<a href="https://colab.research.google.com/github/Deepak-Mewada/Miscellaneous/blob/main/Fashion_MNIST_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#
# Basic Datasets Question
#
# Create a classifier for the Fashion MNIST dataset
# Note that the test will expect it to classify 10 classes and that the
# input shape should be the native size of the Fashion MNIST dataset which is
# 28x28 monochrome. Do not resize the data. Your input layer should accept
# (28,28) as the input shape only. If you amend this, the tests will fail.
#
import tensorflow as tf




In [ ]:
def solution_model():
    fashion_mnist = tf.keras.datasets.fashion_mnist


    
    # YOUR CODE HERE
    return model



In [2]:

# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, your saved .h5 model will
# be sent to the testing infrastructure for scoring
# and the score will be returned to you.
if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")


NameError: ignored

In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)




In [9]:

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(0.2),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=50,
    validation_data=ds_test,
)

model.save("mymodel1.h5")


Epoch 1/50
469/469 [==============================] - 8s 6ms/step - loss: 2.3026 - sparse_categorical_accuracy: 0.0987 - val_loss: 2.3026 - val_sparse_categorical_accuracy: 0.0980
Epoch 2/50
469/469 [==============================] - 2s 5ms/step - loss: 2.3026 - sparse_categorical_accuracy: 0.0987 - val_loss: 2.3026 - val_sparse_categorical_accuracy: 0.0980
Epoch 3/50
469/469 [==============================] - 2s 4ms/step - loss: 2.3026 - sparse_categorical_accuracy: 0.0987 - val_loss: 2.3026 - val_sparse_categorical_accuracy: 0.0980
Epoch 4/50
469/469 [==============================] - 2s 4ms/step - loss: 2.3026 - sparse_categorical_accuracy: 0.0987 - val_loss: 2.3026 - val_sparse_categorical_accuracy: 0.0980
Epoch 5/50
469/469 [==============================] - 2s 4ms/step - loss: 2.3026 - sparse_categorical_accuracy: 0.0987 - val_loss: 2.3026 - val_sparse_categorical_accuracy: 0.0980
Epoch 6/50
469/469 [==============================] - 2s 4ms/step - loss: 2.3026 - sparse_categorica

In [2]:
# ======================================================================
# There are 5 questions in this exam with increasing difficulty from 1-5.
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score significantly
# less than your Category 5 question.
#
# Don't use lambda layers in your model.
# You do not need them to solve the question.
# Lambda layers are not supported by the grading infrastructure.
#
# You must use the Submit and Test button to submit your model
# at least once in this category before you finally submit your exam,
# otherwise you will score zero for this category.
# ======================================================================
#
# Basic Datasets Question
#
# Create and train a classifier for the MNIST dataset.
# Note that the test will expect it to classify 10 classes and that the 
# input shape should be the native size of the MNIST dataset which is 
# 28x28 monochrome. Do not resize the data. Your input layer should accept
# (28,28) as the input shape only. If you amend this, the tests will fail.
#

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential

physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass

class OutLayer(tf.keras.layers.Layer):
    def _init_(self):
        super(OutLayer, self)._init_()
    def call(self, inputs):
        return tf.expand_dims(inputs, 3)

def solution_model():
    mnist = tf.keras.datasets.mnist
    (training_images, training_labels), (test_images, test_labels) = mnist.load_data()

    training_images = training_images / 255.0
    test_images = test_images / 255.0

    callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            min_delta=1e-4,
            patience=3,
            verbose=1
        ),
        ModelCheckpoint(
            filepath='mymodel.h5',
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        )
    ]

    model = Sequential()
    model.add(OutLayer())
    model.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(13, 13, 16)))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(
        training_images,
        training_labels,
        batch_size=128,
        epochs=50,
        verbose=1,
        validation_data=(test_images, test_labels),
        callbacks=callbacks
    )

    # YOUR CODE HERE
    return model


# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, your saved .h5 model will
# be sent to the testing infrastructure for scoring
# and the score will be returned to you.


In [4]:
model = solution_model()
model.save("mymodel.h5")

11490434/11490434 [==============================] - 1s 0us/step
Epoch 1/50
468/469 [============================>.] - ETA: 0s - loss: 0.3696 - accuracy: 0.8857
Epoch 1: val_accuracy improved from -inf to 0.97480, saving model to mymodel.h5
469/469 [==============================] - 26s 54ms/step - loss: 0.3691 - accuracy: 0.8858 - val_loss: 0.0796 - val_accuracy: 0.9748
Epoch 2/50
468/469 [============================>.] - ETA: 0s - loss: 0.1042 - accuracy: 0.9684
Epoch 2: val_accuracy improved from 0.97480 to 0.98250, saving model to mymodel.h5
469/469 [==============================] - 25s 54ms/step - loss: 0.1042 - accuracy: 0.9684 - val_loss: 0.0529 - val_accuracy: 0.9825
Epoch 3/50
468/469 [============================>.] - ETA: 0s - loss: 0.0754 - accuracy: 0.9779
Epoch 3: val_accuracy improved from 0.98250 to 0.98580, saving model to mymodel.h5
469/469 [==============================] - 25s 54ms/step - loss: 0.0753 - accuracy: 0.9779 - val_loss: 0.0434 - val_accuracy: 0.9858
Ep